# ML lab 10 Practice

---

# NN,ReLu,Xavier,Droput,and Adam

> YouTube Lecture: [link](https://www.youtube.com/watch?v=ls8jHqRnEQk)

### Summary

- Softmax Classifier: 90.85%

- Neural Nets: 94.69%

- Xavier Initialization: 97.90%

- Deep Neural Nets **without Dropout**: 97.75% (<font color="red">Overfitting</font>)

- Deep Neural Nets with Dropout: 98.25% (Best)

#### Optimizer

Adam Optimizer is good (Best?)

## Softmax classifier for MNIST

- Accuracy: 90.85%

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)


# Parameters
learning_rate = 0.01
training_epochs = 15
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder("float", [None, 784])  # mnist data image of shape 28 * 28 = 784
y = tf.placeholder("float", [None, 10])  # 0-9 digits recongnition => 10 classes

# Create Model

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
activation = tf.nn.softmax(tf.matmul(x, W) + b)  # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(activation), reduction_indices=1))  # Cross entropy
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)  # Gradient Descent

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys}) / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch: {:<4}, Cost: {:0.9f}".format(epoch+1, avg_cost))
            
    print("Optimization Finished")

    # Test model
    correct_prediction = tf.equal(tf.argmax(activation, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy: {}".format(accuracy.eval({x: mnist.test.images, y: mnist.test.labels})))

---

```
Epoch: 1   , Cost: 1.176996229
Epoch: 2   , Cost: 0.662694740
Epoch: 3   , Cost: 0.550580185
Epoch: 4   , Cost: 0.496816136
Epoch: 5   , Cost: 0.463777680
Epoch: 6   , Cost: 0.440911935
Epoch: 7   , Cost: 0.423966596
Epoch: 8   , Cost: 0.410634728
Epoch: 9   , Cost: 0.399837812
Epoch: 10  , Cost: 0.390884601
Epoch: 11  , Cost: 0.383286627
Epoch: 12  , Cost: 0.376785669
Epoch: 13  , Cost: 0.371010976
Epoch: 14  , Cost: 0.365967462
Epoch: 15  , Cost: 0.361345127
Optimization Finished
Accuracy: 0.9085999727249146
```

---

## Neural Nets (NN) for MNIST

- Accuracy: 94.69%

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)


# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# tf Graph Input
X = tf.placeholder("float", [None, 784])  # mnist data image of shape 28 * 28 = 784
Y = tf.placeholder("float", [None, 10])  # 0-9 digits recongnition => 10 classes

# Stoe layers weight & bias
W1 = tf.Variable(tf.random_normal([784, 256]))
W2 = tf.Variable(tf.random_normal([256, 256]))
W3 = tf.Variable(tf.random_normal([256, 10]))

B1 = tf.Variable(tf.random_normal([256]))
B2 = tf.Variable(tf.random_normal([256]))
B3 = tf.Variable(tf.random_normal([10]))

# Construct Model
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), B1))
L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), B2))  # Hideen layer with RELU activation
hypothesis = tf.add(tf.matmul(L2, W3), B3)

# Minimize error using cross entropy
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))  # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)  # Adam Optimizer

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys})
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={X: batch_xs, Y: batch_ys}) / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch: {:<4}, Cost: {:0.9f}".format(epoch+1, avg_cost))
            
    print("Optimization Finished")

    # Test model
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy: {}".format(accuracy.eval({X: mnist.test.images, Y: mnist.test.labels})))

---

```
Epoch: 1   , Cost: 161.290532403
Epoch: 2   , Cost: 38.226604146
Epoch: 3   , Cost: 23.637878357
Epoch: 4   , Cost: 16.094191647
Epoch: 5   , Cost: 11.488096523
Epoch: 6   , Cost: 8.273973035
Epoch: 7   , Cost: 5.934017671
Epoch: 8   , Cost: 4.425817456
Epoch: 9   , Cost: 3.055886099
Epoch: 10  , Cost: 2.176679079
Epoch: 11  , Cost: 1.519386546
Epoch: 12  , Cost: 1.111432732
Epoch: 13  , Cost: 0.756404809
Epoch: 14  , Cost: 0.583292915
Epoch: 15  , Cost: 0.358361932
Optimization Finished
Accuracy: 0.9469000101089478
```

---

- Softmax classifier 보다 정확도가 상승

### Xavier for MNIST

- Accuracy: 97.90%

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)


# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# tf Graph Input
X = tf.placeholder(tf.float32, [None, 784])  # mnist data image of shape 28 * 28 = 784
Y = tf.placeholder(tf.float32, [None, 10])  # 0-9 digits recongnition => 10 classes

# weights & bias for nn Layers
W1 = tf.get_variable("W1", shape=[784, 256],
                    initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[256, 256],
                    initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[256, 10],
                    initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

# Minimize error using cross entropy
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))  # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)  # Adam Optimizer

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys})
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={X: batch_xs, Y: batch_ys}) / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch: {:<4}, Cost: {:0.9f}".format(epoch+1, avg_cost))
            
    print("Optimization Finished")

    # Test model
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy: {}".format(accuracy.eval({X: mnist.test.images, Y: mnist.test.labels})))

---

```
Epoch: 1   , Cost: 0.273548285
Epoch: 2   , Cost: 0.102229812
Epoch: 3   , Cost: 0.065387963
Epoch: 4   , Cost: 0.044083265
Epoch: 5   , Cost: 0.031710115
Epoch: 6   , Cost: 0.022418242
Epoch: 7   , Cost: 0.016052051
Epoch: 8   , Cost: 0.012779035
Epoch: 9   , Cost: 0.010050793
Epoch: 10  , Cost: 0.008484991
Epoch: 11  , Cost: 0.005212717
Epoch: 12  , Cost: 0.004885615
Epoch: 13  , Cost: 0.004464335
Epoch: 14  , Cost: 0.003929099
Epoch: 15  , Cost: 0.003103349
Optimization Finished
Accuracy: 0.9790999889373779
```

---

- Xavier 초깃값을 사용하여 Accuracy 가 상승

### Deep NN for MNIST

- Accuracy: 97.75% (<font color="red">Overfitting</font>)

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)


# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# tf Graph Input
X = tf.placeholder(tf.float32, [None, 784])  # mnist data image of shape 28 * 28 = 784
Y = tf.placeholder(tf.float32, [None, 10])  # 0-9 digits recongnition => 10 classes

# weights & bias for nn Layers
W1 = tf.get_variable("W1", shape=[784, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)


W4 = tf.get_variable("W4", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

W5 = tf.get_variable("W5", shape=[512, 10],
                    initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))  # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)  # Adam Optimizer

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys})
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={X: batch_xs, Y: batch_ys}) / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch: {:<4}, Cost: {:0.9f}".format(epoch+1, avg_cost))
            
    print("Optimization Finished")

    # Test model
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy: {}".format(accuracy.eval({X: mnist.test.images, Y: mnist.test.labels})))

---

```
Epoch: 1   , Cost: 0.270382025
Epoch: 2   , Cost: 0.077262623
Epoch: 3   , Cost: 0.044846847
Epoch: 4   , Cost: 0.032626795
Epoch: 5   , Cost: 0.021638488
Epoch: 6   , Cost: 0.015611893
Epoch: 7   , Cost: 0.013224729
Epoch: 8   , Cost: 0.010947733
Epoch: 9   , Cost: 0.009347267
Epoch: 10  , Cost: 0.008289753
Epoch: 11  , Cost: 0.007949208
Epoch: 12  , Cost: 0.005896471
Epoch: 13  , Cost: 0.005053920
Epoch: 14  , Cost: 0.005107425
Epoch: 15  , Cost: 0.004195114
Optimization Finished
Accuracy: 0.9775000214576721
```

---

- Layer 를 깊게 만들었으나 Overfitting 발생

- 즉, train 데이터를 지나치게 잘 학습하여  
test 데이터를 오히려 잘 인식하지 못하게 된 경우

- 해결위해 Dropout 을 사용한다.

### Dropout for MNIST

- Accuracy: 98.25% **(BEST)**

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)


# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# tf Graph Input
X = tf.placeholder(tf.float32, [None, 784])  # mnist data image of shape 28 * 28 = 784
Y = tf.placeholder(tf.float32, [None, 10])  # 0-9 digits recongnition => 10 classes

# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable("W1", shape=[784, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[512, 512],
                    initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5", shape=[512, 10],
                    initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))  # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)  # Adam Optimizer

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
            c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            avg_cost += c / total_batch

        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch: {:<4}, Cost: {:0.9f}".format(epoch+1, avg_cost))
            
    print("Optimization Finished")

    # Test model
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy: {}".format(accuracy.eval({X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1})))

---

```
Epoch: 1   , Cost: 0.464075328
Epoch: 2   , Cost: 0.170548030
Epoch: 3   , Cost: 0.133839501
Epoch: 4   , Cost: 0.106930489
Epoch: 5   , Cost: 0.093738453
Epoch: 6   , Cost: 0.082922424
Epoch: 7   , Cost: 0.073176331
Epoch: 8   , Cost: 0.068863922
Epoch: 9   , Cost: 0.062661235
Epoch: 10  , Cost: 0.060985195
Epoch: 11  , Cost: 0.060985552
Epoch: 12  , Cost: 0.053915262
Epoch: 13  , Cost: 0.050093469
Epoch: 14  , Cost: 0.048985790
Epoch: 15  , Cost: 0.042928032
Optimization Finished
Accuracy: 0.9825000166893005
```

---

- Accuracy 가 상승하였다.